In [14]:
import gym
import numpy as np
import tensorflow.compat.v1 as tf
import collections
import os
from datetime import datetime
import time


# optimized for Tf2
tf.disable_v2_behavior()

class PolicyNetwork:
    def __init__(self, state_size, action_size, learning_rate, name='policy_network'):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate

        with tf.variable_scope(name):

            self.state = tf.placeholder(tf.float32, [None, self.state_size], name="state")
            self.gamma_t = tf.placeholder(tf.float32, name="gamma_t")
            self.advantage_delta = tf.placeholder(tf.float32, name="advantage_delta")

            tf2_initializer = tf.keras.initializers.glorot_normal(seed=0)
            self.W1 = tf.get_variable("W1", [self.state_size, 12], initializer=tf2_initializer)
            self.b1 = tf.get_variable("b1", [12], initializer=tf2_initializer)
            self.W2 = tf.get_variable("W2", [12, self.action_size], initializer=tf2_initializer)
            self.b2 = tf.get_variable("b2", [self.action_size], initializer=tf2_initializer)

            self.Z1 = tf.add(tf.matmul(self.state, self.W1), self.b1)
            self.A1 = tf.nn.relu(self.Z1)
            self.output = tf.add(tf.matmul(self.A1, self.W2), self.b2)

            # Softmax probability distribution over actions
            self.actions_distribution = tf.squeeze(tf.nn.softmax(self.output))
            self.actions_log_probs = tf.math.log(self.actions_distribution)

            # Loss calculation  - for gradient ascent we minimize the negative loss. Loss = delta*I*ln(Pi)
            self.loss = self.gamma_t * -tf.math.reduce_sum(self.advantage_delta * self.actions_log_probs)
            self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.loss)

class ValueNetwork:
    def __init__(self, state_size, learning_rate, name='state_value_network'):
        self.state_size = state_size
        self.learning_rate = learning_rate

        with tf.variable_scope(name):

            self.state = tf.placeholder(tf.float32, [None, self.state_size], name="state")
            self.advantage_delta = tf.placeholder(tf.float32, name="advantage_delta")
            self.gamma_t = tf.placeholder(tf.float32, name="gamma_t")


            tf2_initializer = tf.keras.initializers.glorot_normal(seed=0)
            self.W1 = tf.get_variable("W1", [self.state_size, 8], initializer=tf2_initializer)
            self.b1 = tf.get_variable("b1", [8], initializer=tf2_initializer)
            self.W2 = tf.get_variable("W2", [8, 1], initializer=tf2_initializer)
            self.b2 = tf.get_variable("b2", [1], initializer=tf2_initializer)

            self.Z1 = tf.add(tf.matmul(self.state, self.W1), self.b1)
            self.A1 = tf.nn.relu(self.Z1)
            self.output = tf.add(tf.matmul(self.A1, self.W2), self.b2)

            # Loss calculation  - for gradient ascent we minimize the negative loss. Loss = delta*I*V
            self.loss = -self.advantage_delta * self.gamma_t * self.output
            self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.loss)


def run(discount_factor, policy_learning_rate ,sv_learning_rate):
    env = gym.make('CartPole-v1')
    np.random.seed(SEED)
    env.seed(SEED)
    tf.set_random_seed(SEED)
    rewards, mean_rewards, losses = [], [], []
    # Define hyperparameters
    state_size = 4
    action_size = env.action_space.n

    max_episodes = 5000
    max_steps = 501
    discount_factor = discount_factor
    policy_learning_rate = policy_learning_rate
    sv_learning_rate = sv_learning_rate
    render = False

    # Initialize the policy and the state-value network
    tf.reset_default_graph()
    policy = PolicyNetwork(state_size, action_size, policy_learning_rate)
    state_value = ValueNetwork(state_size, sv_learning_rate)

    # Start training the agent with REINFORCE algorithm
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        solved = False
        Transition = collections.namedtuple("Transition", ["state", "action", "reward", "next_state", "done"])
        episode_rewards = np.zeros(max_episodes)
        average_rewards = 0.0
        early_stopping = False

        for episode in range(max_episodes):
            state = env.reset()
            state = state.reshape([1, state_size])
            episode_transitions = []
            since = time.time()
            for step in range(max_steps):
                actions_distribution = sess.run(policy.actions_distribution, {policy.state: state})
                action = np.random.choice(np.arange(len(actions_distribution)), p=actions_distribution)
                next_state, reward, done, _ = env.step(action)
                next_state = next_state.reshape([1, state_size])

                if render:
                    env.render()

                episode_transitions.append(Transition(state=state, action=action, reward=reward, next_state=next_state, done=done))
                episode_rewards[episode] += reward

                if done:
                    if episode > 98:
                        # Check if solved
                        average_rewards = np.mean(episode_rewards[(episode - 99):episode+1])

                    if np.mean(episode_rewards[(episode - 10):episode + 1]) > 475:
                      early_stopping = True

                    print("Episode {} Reward: {} Average over 100 episodes: {}".format(episode, episode_rewards[episode], round(average_rewards, 2)))
                    if average_rewards > 475:
                        print(' Solved at episode: ' + str(episode))
                        time_elapsed = time.time() - since
                        print("Algorithm {} converged after {} seconds".format(algorithm_name, time_elapsed))
                        solved = True
                    break
                state = next_state

            if solved:
                break

            # Compute Gt for each time-step t and update the network's weights
            for t, transition in enumerate(episode_transitions):
                # Compute Gt
                total_discounted_return = sum(discount_factor ** i * t.reward for i, t in enumerate(episode_transitions[t:]))

                # Compute V(S,w)
                feed_dict = {state_value.state: transition.state}
                V_s = sess.run(state_value.output, feed_dict)

                # Compute delta = Gt - V(S,w)
                advantage_delta = total_discounted_return - V_s

                # Update the state_value network weights
                feed_dict = {state_value.state: transition.state, state_value.gamma_t: discount_factor**t,
                             state_value.advantage_delta: advantage_delta}
                _, loss_state = sess.run([state_value.optimizer, state_value.loss], feed_dict)

                # Update the policy network weights
                feed_dict = {policy.state: transition.state, policy.gamma_t: discount_factor**t,
                             policy.advantage_delta: advantage_delta}

                if early_stopping:
                # Early stopping to prevent the network weights from changing after it is stable
                  loss_policy = sess.run(policy.loss, feed_dict)
                else:
                  _, loss_policy = sess.run([policy.optimizer, policy.loss], feed_dict)


            rewards.append(episode_rewards[episode])
            mean_rewards.append(average_rewards)
            losses.append(loss_policy)
    return episode, rewards, mean_rewards, losses


if __name__ == '__main__':
    SEED = 42
    optimal_sv_lr = 0.0007
    optimal_policy_lr = 0.0005
    # global_step = tf.Variable(0, trainable=False)

    # optimal_policy_lr = tf.train.exponential_decay(optimal_policy_lr,
    #                                     global_step, 300,
    #                                     0.95, staircase=True)
    optimal_df = 0.99
    algorithm_name = "reinforce_wt_baseline"
    for df in [0.9 ,0.95 ,0.99 ,0.995 ,0.9995]:
        print("df: ", df)
        last_episode, rewards, mean_rewards, losses = run(discount_factor=0.99, policy_learning_rate=optimal_policy_lr, sv_learning_rate=optimal_sv_lr)
        with open('df_sim_{}_{}.npy'.format(algorithm_name, df), 'wb') as f:
            np.save(f, last_episode)
            np.save(f, rewards)
            np.save(f, mean_rewards)
            np.save(f, losses)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym

df:  0.9


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Episode 0 Reward: 16.0 Average over 100 episodes: 0.0
Episode 1 Reward: 28.0 Average over 100 episodes: 0.0
Episode 2 Reward: 17.0 Average over 100 episodes: 0.0
Episode 3 Reward: 40.0 Average over 100 episodes: 0.0
Episode 4 Reward: 22.0 Average over 100 episodes: 0.0
Episode 5 Reward: 28.0 Average over 100 episodes: 0.0
Episode 6 Reward: 29.0 Average over 100 episodes: 0.0
Episode 7 Reward: 24.0 Average over 100 episodes: 0.0
Episode 8 Reward: 30.0 Average over 100 episodes: 0.0
Episode 9 Reward: 34.0 Average over 100 episodes: 0.0
Episode 10 Reward: 14.0 Average over 100 episodes: 0.0
Episode 11 Reward: 15.0 Average over 100 episodes: 0.0
Episode 12 Reward: 19.0 Average over 100 episodes: 0.0
Episode 13 Reward: 19.0 Average over 100 episodes: 0.0
Episode 14 Reward: 39.0 Average over 100 episodes: 0.0
Episode 15 Reward: 47.0 Average over 100 episodes: 0.0
Episode 16 Reward: 17.0 Average over 100 episodes: 0.0
Episode 17 Reward: 51.0 Average over 100 episodes: 0.0
Episode 18 Reward: 2

KeyboardInterrupt: 